In [ ]:
!pip install joblib tqdm clickhouse-driver pandas numpy statsmodels seaborn matplotlib

In [16]:
from datetime import datetime, timedelta
from clickhouse_driver import Client
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

DATE_CONFIG = {
    'TRAIN_START': datetime(2015, 2, 2).date(),
    'TRAIN_END': datetime(2020, 2, 2).date(),
    'TEST_END': datetime(2025, 1, 1).date(),
    'TRADING_DAYS_PER_YEAR': 252  
}

def get_training_period():
    return {
        'start': DATE_CONFIG['TRAIN_START'],
        'end': DATE_CONFIG['TRAIN_END']
    }

def get_test_period():
    return {
        'start': DATE_CONFIG['TRAIN_END'],
        'end': DATE_CONFIG['TEST_END']
    }

def get_training_days():
    years = (DATE_CONFIG['TRAIN_END'] - DATE_CONFIG['TRAIN_START']).days / 365
    return int(years * DATE_CONFIG['TRADING_DAYS_PER_YEAR'])

SELECTED_SECTOR = 'Financials'

#### Hauptfunktion
Die Hauptfunktion find_cointegrated_pairs():
Diese Funktion ist das Herzstück der Analyse. Sie erstellt zwei Matrizen:
- Eine für Statistik-Scores (score_matrix)
- Eine für p-Werte (pvalue_matrix)

Der Code durchläuft dann jedes mögliche Aktienpaar und führt einen Kointegrationstest durch.

**Zeitbereich:**
Im Code wird der Zeitbereich so festgelegt:
```python
start_date = client.execute('SELECT MIN(date) FROM stock_data')[0][0]
end_date = start_date + timedelta(days=48*30)
```
Das bedeutet, der Code nimmt das früheste verfügbare Datum in der Datenbank und analysiert von dort aus die nächsten 48 Monate (also 4 Jahre). Die Analyse läuft also über einen 4-Jahres-Zeitraum, beginnend mit dem ersten verfügbaren Datenpunkt.

1. Teststatistik:
Stellen Sie sich die Teststatistik wie ein Thermometer vor. Sie misst, wie stark zwei Aktien miteinander verbunden sind. Je negativer der Wert ist, desto stärker ist die Verbindung. Das ist wie bei einem Thermometer, das unter Null fällt - je weiter unter Null, desto "kälter" oder in unserem Fall, desto stärker verbunden sind die Aktien.

2. P-Wert:
Der P-Wert ist wie eine Wahrscheinlichkeitsangabe auf einer Skala von 0 bis 1. Er sagt uns, wie verlässlich unsere Beobachtung ist:
- Ein p-Wert von 0.05 bedeutet eine 5% Chance, dass wir uns irren
- Je kleiner der p-Wert, desto sicherer können wir sein
- Im Code werden Aktienpaare mit p-Wert < 0.05 als bedeutsam eingestuft
Das ist wie bei einer Wettervorhersage: Wenn die Regenwahrscheinlichkeit bei 5% liegt, sind wir ziemlich sicher, dass es nicht regnen wird.

3. Kritische Werte:
Die kritischen Werte sind wie Grenzlinien. Sie helfen uns zu entscheiden, ob die Teststatistik bedeutsam ist:
- Es gibt typischerweise drei kritische Werte (1%, 5% und 10% Niveau)
- Wenn unsere Teststatistik kleiner (negativer) ist als diese Werte, haben wir einen signifikanten Fund
Das ist wie beim Hochsprung: Die kritischen Werte sind wie verschiedene Höhen der Latte. Wenn ein Springer darüber kommt, ist es eine bedeutende Leistung.

Ein praktisches Beispiel:
Nehmen wir an, wir analysieren zwei Energieaktien:
- Teststatistik: -3.5
- P-Wert: 0.02
- Kritische Werte: [-3.4, -2.9, -2.6]

Das würde bedeuten:
- Die Teststatistik (-3.5) ist kleiner als der strengste kritische Wert (-3.4)
- Der p-Wert (0.02 oder 2%) ist kleiner als 0.05 (5%)
- Schlussfolgerung: Diese Aktien haben eine sehr starke, statistisch signifikante Verbindung

Diese Analyse ist besonders wichtig für Händler, die Pairs-Trading-Strategien entwickeln wollen, wo sie auf die Annäherung von zeitweise auseinandergelaufenen, aber grundsätzlich verbundenen Aktien setzen.

Möchten Sie, dass ich einen dieser Aspekte noch genauer erkläre oder sollen wir uns ansehen, wie diese Werte praktisch interpretiert werden können?

----

#### Visualisierung:
```python
plt.figure(figsize=(12, 8))
mask = (pvalues >= 0.98)
sns.heatmap(pvalues, 
            xticklabels=data.columns, 
            yticklabels=data.columns, 
            cmap='RdYlGn_r',
            mask=mask)
```
Die Heatmap zeigt p-Werte für alle Aktienpaare:
- Die Maske blendet sehr hohe p-Werte (≥ 0.98) aus
- Die Farbskala geht von Rot (hohe p-Werte, keine Kointegration) zu Grün (niedrige p-Werte, starke Kointegration)
- Die Achsen zeigen die Aktiensymbole

In [15]:
def find_cointegrated_pairs(data):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.keys()
    pairs = []

    total_iterations = sum(range(n))

    with tqdm(total=total_iterations, desc="Analyzing pairs") as pbar:
        for i in range(n):
            for j in range(i+1, n):
                S1 = data[keys[i]]
                S2 = data[keys[j]]
                result = coint(S1, S2)
                score = result[0]
                pvalue = result[1]
                score_matrix[i, j] = score
                pvalue_matrix[i, j] = pvalue
                if pvalue <= 0.01:
                    pairs.append((keys[i], keys[j]))
                pbar.update(1)

    return score_matrix, pvalue_matrix, pairs

client = Client(host='localhost', port=9000, user='default', password='password')


In [17]:
def get_sector_pairs():
    print(f"Fetching pairs for sector: {SELECTED_SECTOR if SELECTED_SECTOR else 'ALL'}")
    query = '''
    SELECT pair_key, sector, symbol1, symbol2 
    FROM stock_pairs
    '''
    if SELECTED_SECTOR:
        query += f" WHERE sector = '{SELECTED_SECTOR}'"
    pairs_df = pd.DataFrame(client.execute(query), 
                          columns=['pair_key', 'sector', 'symbol1', 'symbol2'])
    print(f"Found {len(pairs_df)} pairs")
    return pairs_df

def get_stock_data(symbols):
    placeholders = ', '.join(f"'{s}'" for s in symbols)
    query = f'''
    SELECT symbol, date, close 
    FROM stock_data
    WHERE symbol IN ({placeholders})
    AND date BETWEEN '{DATE_CONFIG['TRAIN_START']}' AND '{DATE_CONFIG['TEST_END']}'
    ORDER BY symbol, date
    '''
    print("Loading stock data...")
    df = pd.DataFrame(
        client.execute(query), 
        columns=['symbol', 'date', 'close']
    )
    print(f"Loaded data for {len(symbols)} symbols")
    return df.pivot(columns='symbol', values='close', index='date')

In [ ]:
print("Starting analysis...")
pairs_df = get_sector_pairs()
unique_symbols = pd.concat([pairs_df['symbol1'], pairs_df['symbol2']]).unique()
data = get_stock_data(unique_symbols)

print("Cleaning data...")
min_periods = len(data) * 0.95

print(f"Analysis will be performed on {data.shape[1]} symbols")

print("Starting cointegration analysis...")
scores, pvalues, pairs = find_cointegrated_pairs(data)

# %%
print("Creating visualization...")
plt.figure(figsize=(12, 8))
mask = (pvalues >= 0.98)
sns.heatmap(pvalues, 
            xticklabels=data.columns, 
            yticklabels=data.columns, 
            cmap='RdYlGn_r',
            mask=mask)
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title('Cointegration p-values heatmap')
plt.tight_layout()
plt.show()

print("\nCointegrated Pairs (p-value < 0.01):")
for pair in pairs:
    print(f"{pair[0]} - {pair[1]}")

print("\nAnalysis complete!")

In [ ]:
def plot_pairs(data, pairs):
    for s1, s2 in pairs:
        S1 = data[s1]
        S2 = data[s2]
        score, pvalue, _ = coint(S1, S2)
        ratios = S1 / S2
        
        plt.figure(figsize=(15,7))
        ratios.plot()
        plt.axhline(ratios.mean(), color='r')
        plt.title(f'{s1} / {s2} Price Ratio (p-value: {pvalue:.4f})')
        plt.legend(['Price Ratio', 'Mean'])
        plt.show()
        print(f"\n{s1} - {s2} p-value: {pvalue}")

plot_pairs(data, pairs)

In [ ]:
def zscore(series):
    return (series - series.mean()) / np.std(series)

for pair in pairs:
    symbol1, symbol2 = pair
    
    ratios = data[symbol1] / data[symbol2]
    
    plt.figure(figsize=(15,7))
    zscore(ratios).plot()
    plt.axhline(zscore(ratios).mean(), color='black')
    plt.axhline(1.0, color='red', linestyle='--')
    plt.axhline(-1.0, color='green', linestyle='--')
    plt.title(f'Z-Score: {symbol1} vs {symbol2}')
    plt.legend(['Ratio z-score', 'Mean', '+1', '-1'])
    plt.show()

In [ ]:
for pair in pairs:
    symbol1, symbol2 = pair
    
    query = f"""
    WITH pair_data AS (
        SELECT symbol, date, close 
        FROM stock_data
        WHERE symbol IN ('{symbol1}', '{symbol2}')
        AND date BETWEEN '{DATE_CONFIG['TRAIN_START']}' AND '{DATE_CONFIG['TEST_END']}'
        ORDER BY symbol, date
    )
    SELECT * FROM pair_data
    """
    
    df = pd.DataFrame(
        client.execute(query),
        columns=['symbol', 'date', 'close']
    )
    
    pair_data = df.pivot(columns='symbol', values='close', index='date')
    ratios = pair_data[symbol1] / pair_data[symbol2]
    training_days = get_training_days()
    train = ratios[:training_days]
    test = ratios[training_days:]
    
    ratios_mavg5 = train.rolling(window=5, center=False).mean()
    ratios_mavg60 = train.rolling(window=60, center=False).mean()
    std_60 = train.rolling(window=60, center=False).std()
    zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60
    
    plt.figure(figsize=(15,7))
    plt.plot(train.index, train.values)
    plt.plot(ratios_mavg5.index, ratios_mavg5.values)
    plt.plot(ratios_mavg60.index, ratios_mavg60.values)
    
    plt.title(f'Moving Averages: {symbol1} vs {symbol2}')
    plt.legend(['Ratio','5d Ratio MA', '60d Ratio MA'])
    plt.ylabel('Ratio')
    plt.show()

In [ ]:
for pair in pairs:
    symbol1, symbol2 = pair
    
    query = f"""
    WITH pair_data AS (
        SELECT symbol, date, close 
        FROM stock_data
        WHERE symbol IN ('{symbol1}', '{symbol2}')
        AND date BETWEEN '{DATE_CONFIG['TRAIN_START']}' AND '{DATE_CONFIG['TEST_END']}'
        ORDER BY symbol, date
    )
    SELECT * FROM pair_data
    """
    
    df = pd.DataFrame(
        client.execute(query),
        columns=['symbol', 'date', 'close']
    )
    
    pair_data = df.pivot(columns='symbol', values='close', index='date')
    ratios = pair_data[symbol1] / pair_data[symbol2]
    training_days = get_training_days()
    train = ratios[:training_days]
    
    ratios_mavg5 = train.rolling(window=5, center=False).mean()
    ratios_mavg60 = train.rolling(window=60, center=False).mean()
    std_60 = train.rolling(window=60, center=False).std()
    zscore_60_5 = (ratios_mavg5 - ratios_mavg60)/std_60
    
    buy = train.copy()
    sell = train.copy()
    buy[zscore_60_5>-1] = 0
    sell[zscore_60_5<1] = 0
    
    plt.figure(figsize=(18,9))
    S1 = pair_data[symbol1].iloc[:training_days]
    S2 = pair_data[symbol2].iloc[:training_days]
    
    S1[60:].plot(color='b')
    S2[60:].plot(color='c')
    
    buyR = 0*S1.copy()
    sellR = 0*S1.copy()
    
    buyR[buy!=0] = S1[buy!=0]
    sellR[buy!=0] = S2[buy!=0]
    buyR[sell!=0] = S2[sell!=0]
    sellR[sell!=0] = S1[sell!=0]
    
    buyR[60:].plot(color='g', linestyle='None', marker='^')
    sellR[60:].plot(color='r', linestyle='None', marker='^')
    
    x1,x2,y1,y2 = plt.axis()
    plt.axis((x1,x2,min(S1.min(),S2.min()),max(S1.max(),S2.max())))
    
    plt.title(f'Price Action: {symbol1} vs {symbol2}')
    plt.legend([symbol1, symbol2, 'Buy Signal', 'Sell Signal'])
    plt.show()

1. **Ratios (ratios_train = S1_train/S2_train)**
- **Bedeutung**: Zeigt die relative Preisbeziehung zwischen zwei Aktien
- **Eigenschaft**: Sollte theoretisch um einen Mittelwert schwanken 
- **Nutzen**: Identifiziert, wenn Aktien aus ihrer historischen Preisbeziehung ausbrechen
- **Beispiel**: Wenn zwei Banken normalerweise im Verhältnis 2:1 handeln und plötzlich auf 2.5:1 steigen, könnte dies eine Handelsmöglichkeit sein

2. **Moving Averages (ma1_train, ma2_train)**
- **Kurzer MA (5 Tage)**:
  - **Eigenschaft**: Reagiert schnell auf Preisänderungen
  - **Nutzen**: Zeigt kurzfristige Trends
  - **Aussage**: Hilft kurzfristige Abweichungen zu erkennen

- **Langer MA (60 Tage)**:
  - **Eigenschaft**: Glättet Preisschwankungen stark
  - **Nutzen**: Bestimmt den langfristigen "normalen" Zustand
  - **Aussage**: Dient als Referenzpunkt für "faire" Bewertung

3. **Standardabweichung (std_train)**
- **Bedeutung**: Misst die "normale" Schwankungsbreite der Ratios
- **Eigenschaft**: Höhere Werte bedeuten mehr Volatilität
- **Nutzen**: Hilft zu bestimmen, ob eine Abweichung signifikant ist
- **Beispiel**: Wenn Ratio normalerweise ±5% schwankt, ist eine 15% Abweichung bedeutend

4. **Z-Score**
- **Bedeutung**: Standardisierte Abweichung vom Durchschnitt
- **Eigenschaften**:
  - \>1: Stark überdurchschnittlich
  - <-1: Stark unterdurchschnittlich
  - Zwischen -0.5 und 0.5: "Normal"
- **Nutzen**: Automatisierte Handelsentscheidungen
- **Beispiel**: 
  - Z-Score = 2 bedeutet: Ratio ist 2 Standardabweichungen über normal
  - Deutet auf überbewertet/unterbewertet hin

5. **Position Tracking (countS1, countS2)**
- **Bedeutung**: Aktuelle Handelsposition
- **Eigenschaften**: 
  - Positiv: Long-Position
  - Negativ: Short-Position
- **Nutzen**: Verfolgt offene Positionen und deren Größe
- **Beispiel**: 
  - countS1 = -1, countS2 = +2 bedeutet:
  - Short 1 Einheit von S1
  - Long 2 Einheiten von S2

6. **Money (Gewinn/Verlust)**
- **Bedeutung**: Kumulierter Handelserfolg
- **Eigenschaft**: Summe aller realisierten Gewinne/Verluste
- **Nutzen**: Misst Strategie-Performance
- **Beispiel**: 
  - Positive Werte: Profitable Strategie
  - Negative Werte: Verlustbringende Strategie

Die Strategie basiert auf der Annahme, dass extreme Abweichungen (gemessen durch Z-Score) sich wieder normalisieren werden. Wenn der Z-Score extrem wird (>1 oder <-1), wird eine Position aufgebaut, und wenn sich das Verhältnis normalisiert (Z-Score zwischen -0.5 und 0.5), wird die Position geschlossen.

In [23]:
def trade(S1_train, S2_train, S1_test, S2_test, window1, window2):
    ratios_train = S1_train/S2_train
    ma1_train = ratios_train.rolling(window=window1, center=False).mean()
    ma2_train = ratios_train.rolling(window=window2, center=False).mean()
    std_train = ratios_train.rolling(window=window2, center=False).std()
    
    ratios_test = S1_test/S2_test
    
    last_ma1 = ma1_train.iloc[-1]
    last_ma2 = ma2_train.iloc[-1]
    last_std = std_train.iloc[-1]
    
    money = 0
    countS1 = 0
    countS2 = 0
    for i in range(len(ratios_test)):
        current_ratio = ratios_test.iloc[i]
        zscore = (current_ratio - last_ma2)/last_std
        
        if zscore > 1:
            money += S1_test.iloc[i] - S2_test.iloc[i] * current_ratio
            countS1 -= 1
            countS2 += current_ratio
        elif zscore < -1:
            money -= S1_test.iloc[i] - S2_test.iloc[i] * current_ratio
            countS1 += 1
            countS2 -= current_ratio
        elif abs(zscore) < 0.5:
            money += countS1*S1_test.iloc[i] + S2_test.iloc[i] * countS2
            countS1 = 0
            countS2 = 0
            
        last_ma1 = 0.8 * last_ma1 + 0.2 * current_ratio
        last_ma2 = 0.983 * last_ma2 + 0.017 * current_ratio
        
    return money

In [29]:
print("Trading Results");
print(DATE_CONFIG['TRAIN_START'])
print("to");
print(DATE_CONFIG['TEST_END'])

for pair in pairs:
    symbol1, symbol2 = pair
    
    query = f"""
    WITH pair_data AS (
        SELECT symbol, date, close 
        FROM stock_data
        WHERE symbol IN ('{symbol1}', '{symbol2}')
        AND date BETWEEN '{DATE_CONFIG['TRAIN_START']}' AND '{DATE_CONFIG['TEST_END']}'
        ORDER BY symbol, date
    )
    SELECT * FROM pair_data
    """
    
    df = pd.DataFrame(
        client.execute(query),
        columns=['symbol', 'date', 'close']
    )
    
    pair_data = df.pivot(columns='symbol', values='close', index='date')
    
    training_mask = pair_data.index < DATE_CONFIG['TRAIN_END']
    
    S1_train = pair_data[symbol1][training_mask]
    S2_train = pair_data[symbol2][training_mask]
    S1_test = pair_data[symbol1][~training_mask]
    S2_test = pair_data[symbol2][~training_mask]
    
    profit = trade(S1_train, S2_train, S1_test, S2_test, 5, 60)
    print(f"{symbol1} - {symbol2}: ${profit:.2f}")

Trading Results
2015-02-02
to
2025-01-01
AFL - TRV: $651.42
AIZ - BX: $-81.60
ALL - AXP: $5788.89
ALL - JPM: $2046.50
AMP - TRV: $7849.42
AON - FDS: $7624.89
APO - ERIE: $1902.58
APO - PGR: $2136.20
AXP - TRV: $6355.39
BAC - PNC: $582.75
BEN - CPAY: $725.16
BRK.B - CB: $9095.72
BRK.B - TRV: $5474.34
CB - MET: $4241.67
CB - RJF: $4568.26
CB - TRV: $2700.47
CFG - NTRS: $929.29
CINF - DFS: $4710.33
DFS - GS: $2127.99
DFS - TRV: $3777.53
ERIE - FI: $12513.19
ERIE - PGR: $5342.43
FITB - PNC: $602.23
GPN - MKTX: $3016.43
HBAN - PNC: $187.53
ICE - MCO: $1328.96
ICE - WTW: $3076.20
KEY - NTRS: $495.95
MA - V: $5922.47
MA - WTW: $11798.86
MCO - WTW: $11105.21
MET - RJF: $1477.54
MET - TRV: $1444.15
NTRS - PNC: $1898.98
NTRS - TFC: $2549.02
RF - SCHW: $448.89
RJF - TRV: $2973.27
SPGI - WTW: $9874.49
V - WTW: $7038.66
